In [1]:
import cv2
import numpy as np 

In [2]:
def nothing(x):
    # any operation
    pass

In [40]:
def rescale_frame(frame_input, percent=50):
    width = int(frame_input.shape[1] * percent / 100)
    height = int(frame_input.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame_input, dim, interpolation=cv2.INTER_AREA)

In [119]:
# cap = cv2.VideoCapture(1)
# cap = cv2.VideoCapture("IMG_1178.MOV")
cap = cv2.VideoCapture("IMG_1524.MOV")

cv2.namedWindow("Trackbars")
cv2.createTrackbar("thre1", "Trackbars", 30, 255, nothing)
cv2.createTrackbar("thre2", "Trackbars", 20, 255, nothing)
cv2.createTrackbar("thre3", "Trackbars", 1000, 2000, nothing)
cv2.createTrackbar("thre4", "Trackbars", 1, 10, nothing)

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    _, frame = cap.read()
    if frame is None:
        break

    frame = rescale_frame(frame)

    thre1 = cv2.getTrackbarPos("thre1", "Trackbars")
    thre2 = cv2.getTrackbarPos("thre2", "Trackbars")
    thre3 = cv2.getTrackbarPos("thre3", "Trackbars")
    thre4 = cv2.getTrackbarPos("thre4", "Trackbars")
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    dilated_img = cv2.dilate(blurred_frame, np.ones((7, 7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = cv2.absdiff(gray_frame, bg_img)
    norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    canny_frame = cv2.Canny(norm_img, thre1, thre2)
    canny_frame = cv2.dilate(canny_frame, kernel=np.ones((3, 3), np.uint8), iterations=1)
    canny_frame = cv2.erode(canny_frame, kernel=np.ones((3, 3), np.uint8), iterations=1)

    mask1 = np.ones(frame.shape, dtype=np.uint8)
    contours1, _ = cv2.findContours(canny_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt1 in contours1:
        area = cv2.contourArea(cnt1)
        approx = cv2.approxPolyDP(cnt1, (thre4/100.0)*cv2.arcLength(cnt1, True), True)

        if area >= thre3:
            cv2.drawContours(mask1, [approx], 0, (255, 255, 255), 2)
            cv2.fillPoly(mask1, [approx], (255, 255, 255))

    cv2.imshow("Frame", frame)
    cv2.imshow("norm_img", norm_img)
    cv2.imshow("Edge", canny_frame)
    cv2.imshow("Mask1", mask1)
    if cv2.waitKey(30) == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()